## **<span style="color:blue">University of Management & Technology</span>**
### <span>Deep Learning & Neural Networks</span>
### Assignment 2

<table style="width:100%; border-collapse:collapse; margin-top:20px; font-family:Arial, sans-serif; border:1px solid #bbb;">

  <tr>
    <th colspan="2" 
        style="background:#0056b3; color:white; text-align:left; 
               padding:10px; font-size:18px; font-weight:bold; 
               border-bottom:1px solid #bbb;">
      Information
    </th>
  </tr>

  <tr>
    <td style="padding:10px; font-weight:bold; width:30%;">Student Name:</td>
    <td style="padding:10px;">Syed Shahzil Abbas</td>
  </tr>

  <tr>
    <td style="padding:10px; font-weight:bold;">ID:</td>
    <td style="padding:10px;">23018020020</td>
  </tr>

  <tr>
    <td style="padding:10px; font-weight:bold;">Program:</td>
    <td style="padding:10px;">BSCS Sec A Batch 18 — Semester Fall 25</td>
  </tr>

  <tr>
    <td style="padding:10px; font-weight:bold;">Course Title:</td>
    <td style="padding:10px;">Deep Learning and Neural Networks</td>
  </tr>

  <tr>
    <td style="padding:10px; font-weight:bold;">Resource Person:</td>
    <td style="padding:10px;">Ms Hina Tufail</td>
  </tr>
</table>

**_<spam style="color:blue">KeyPoints:</span>_**
- Load the same dataset used in Logistic Regression.
- init_param(n_x, n_h, n_y).
- Activation functions: sigmoid, relu.
- Forward Propagation. 
- Cost Function.
- Backword Propagation.
- Fit (Gradient Descent).
- Analysis Report 


**_Loading and Spliting Dataset_**

In [26]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split


In [27]:
data = pd.read_csv("Bank_Customer_Churn_Prediction_Processed.csv")
X =data[['country_Spain','country_Germany','gender_Male','credit_card','active_member','age','tenure','credit_score','balance','products_number','estimated_salary']]
Y = data['churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)


In [28]:
# Shape of inputs
xtrain_shape =X_train.shape
ytrain_shape =Y_train.shape
xtest_shape  =X_test.shape
ytest_shape  =Y_test.shape
m_train = X_train.shape[0]
m_test = X_test.shape[0]

print("X_train shape:", xtrain_shape)
print("Y_train shape:", ytrain_shape)
print("X_test shape:", xtest_shape)
print("Y_test shape:", ytest_shape)
print("Number of training examples:", m_train)
print("Number of testing examples:", m_test)



X_train shape: (8000, 11)
Y_train shape: (8000,)
X_test shape: (2000, 11)
Y_test shape: (2000,)
Number of training examples: 8000
Number of testing examples: 2000


In [29]:
# Layer Sizes
n_x = X_train.shape[1] 
n_h = 4                 
n_y = 1 

**_Initialize Parameters_**

In [30]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """    
    W1=np.random.randn(n_h,n_x) *0.01
    W2=np.random.randn(n_y,n_h) *0.01
    b1=np.zeros((n_h,1))
    b2=np.zeros((n_y,1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

**_Activation Function:_**
$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$
$$
\text{ReLU}(z) = \max(0, z)
$$

In [31]:
def sigmoid(z):
    z = np.clip(z, -500, 500)
    s = 1/(1+np.exp(-z))
    return s
def relu(z):
    r = np.maximum(0,z)
    return r

**_Forward Propagation:_**

$$
Z^{[1]} = W^{[1]} X + b^{[1]} \tag{1}
$$
$$
A^{[1]} = \text{ReLU}\left(Z^{[1]}\right) \tag{2}
$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}\tag{3}$$
$$\hat{Y} = A^{[2]} = \sigma(Z^{[2]})\tag{4}$$



In [32]:
def forward_propagation(x, parameters):
    """
    Argument:
    x -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    # LAYER 1
    Z1=np.dot(W1,x)+b1
    A1=relu(Z1)
    # LAYER 2
    Z2=np.dot(W2,A1)+b2
    A2=sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

**_Cost Function_**
- Cross Entropy 
$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small$$


In [33]:
def compute_cost(A2, y):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    y -- "true" labels vector of shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    if(y.ndim == 1):
        m = y.shape[0] # number of examples
    else:
        m = y.shape[1] # number of examples
    logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
    cost= -np.sum(logprobs)/m
    
    cost = float(np.squeeze(cost))   
                                    
    return cost

**_Backword Propagation_**

$$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$$

$$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $$

$$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$$

$$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $$

$$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $$

$$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$$


In [34]:
def backward_propagation(parameters, cache, x, y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    x -- input data of shape (2, number of examples)
    y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = x.shape[1]
    
    W1=parameters["W1"]
    W2=parameters["W2"]
        
    A1=cache["A1"]
    A2=cache["A2"]
    
    dZ2=A2 - y
    dW2=(1/m)*np.dot(dZ2,A1.T)
    db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1=np.dot(W2.T,dZ2)*(1 - np.power(A1,2))
    dW1=(1/m)*np.dot(dZ1,x.T)
    db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

**_Fit Function_**
- Performs gradient descent.


In [35]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    
    dW1=grads["dW1"]
    db1=grads["db1"]
    dW2=grads["dW2"]
    db2=grads["db2"]
    
    W1=W1 - learning_rate * dW1
    b1=b1 - learning_rate * db1
    W2=W2 - learning_rate * dW2
    b2=b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [36]:
def fit(x, y, n_h, l_r, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    x -- dataset of shape (2, number of examples)
    y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    l_r -- learning rate
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    n_x = x.shape[0]
    n_y = 1  
    parameters=initialize_parameters(n_x,n_h,n_y)
    
    for i in range(0, num_iterations):
        A2,cache=forward_propagation(x,parameters)
        cost=compute_cost(A2,y)
        grads=backward_propagation(parameters,cache,x,y)
        parameters=update_parameters(parameters,grads, l_r)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

**Analytics**

In [40]:
parameters = fit(X_train.T, Y_train.values.reshape(1, -1), n_h = 4, l_r=0.01 ,num_iterations = 9000, print_cost=True)


Cost after iteration 0: 0.693181
Cost after iteration 1000: 0.513927
Cost after iteration 2000: 0.510031
Cost after iteration 3000: 0.505716
Cost after iteration 4000: 0.488122
Cost after iteration 5000: 0.461681
Cost after iteration 6000: 0.452370
Cost after iteration 7000: 0.449463


C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: RuntimeWarning: divide by zero encountered in log
  logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: RuntimeWarning: invalid value encountered in multiply
  logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: RuntimeWarning: divide by zero encountered in log
  logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: RuntimeWarning: invalid value encountered in multiply
  logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: RuntimeWarning: divide by zero encountered in log
  logprobs=np.multiply(np.log(A2),y)+np.multiply(np.log(1-A2),(1-y))
C:\Users\F1C\AppData\Local\Temp\ipykernel_20232\2426020177.py:17: R

Cost after iteration 8000: nan


In [41]:
y_hat, cache =forward_propagation(X_train.T,parameters)
y_hat= 1 if y_hat>=0.5 else 0
print(y_hat) 

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()